Time Series of US Deposit and Swap Rates
========================================

This notebook demonstrates how to download time series of USD deposit and swap rates from the US Federal Reserve Board web site. 

The data is obtained from the site www.federalreserve.gov. The time series can be downloaded from a web browser, but they can also be downloaded programmatically. The site provides directions on how to construct the URL corresponding to each particular data set. In the example below, the URL is specific to the H15 table, with all available deposit and swap rates included.

In this notebook, we download 11 years of daily data, from January 2000 to December 2011. The data is stored in a pandas DataFrame for further processing.

In [1]:
import os, urllib, datetime, pandas
import numpy as np
import math

from pandas.io.parsers import read_csv
from datetime import date

## Utility functions

In [2]:
def get_frb_url(dtStart, dtEnd):
    """
    Federal Reserve Board URL
    Construct this URL at 'http://www.federalreserve.gov/datadownload
    """
    
    url = 'http://www.federalreserve.gov/datadownload/Output.aspx?rel=H15&series=8f47c9df920bbb475f402efa44f35c29&lastObs=&from=%s&to=%s&filetype=csv&label=include&layout=seriescolumn' % (dtStart.strftime('%m/%d/%Y'), dtEnd.strftime('%m/%d/%Y'))
    return url

def dataconverter(s):
    """
    The FRB data file has 
    - numeric cells
    - empty cells
    - cells with 'NC' or 'ND'
    """
    try:
        res = float(s)
    except:
        res = np.nan
    return res

def good_row(z):
    """
    Retain days with no gaps (0 or NaN) in data
    """

    try:
        res = not ((z.isnull()) | (z == 0)).any()
    except:
        res = False
    return res

## Download or read the data, convert to data frame and save  

In [4]:

fname = os.path.join('..', 'data', 'frb_h15.csv')

if not os.path.isfile(fname):
    url = get_frb_url(dtStart=date(2000,1,1),
                          dtEnd=date(2011,12,20))
    frb_site = urllib.urlopen(url)
    text = frb_site.read().strip()

    f = open(fname, 'w')
    f.write(text)
    f.close()

# simpler labels
columns_dic = {"RIFLDIY01_N.B":'Swap1Y',
               "RIFLDIY02_N.B":'Swap2Y',
               "RIFLDIY03_N.B":'Swap3Y',
               "RIFLDIY04_N.B":'Swap4Y',
               "RIFLDIY05_N.B":'Swap5Y',
               "RIFLDIY07_N.B":'Swap7Y',
               "RIFLDIY10_N.B":'Swap10Y',
               "RIFLDIY30_N.B":'Swap30Y',
               "RILSPDEPM01_N.B":'Libor1M',
               "RILSPDEPM03_N.B":'Libor3M',
               "RILSPDEPM06_N.B":'Libor6M'}

# the data converter is applied to all columns
# excluding the index column (0)

dc_dict = {i: dataconverter for i
        in range(1,len(columns_dic) +1)}

# read the csv file and convert to a DataFrame

df_libor = read_csv(fname, sep=',', header=0,
                    index_col=0, parse_dates=True,
                    converters=dc_dict,
                    skiprows=[0,1,2,3,4])

# change column names, remove rows with NaN and save data frame
    
df_libor = df_libor.rename(columns=columns_dic)
good_rows = df_libor.apply(good_row, axis=1)
df_libor_good = df_libor[good_rows]
print(df_libor_good)

df_libor_good.to_pickle(os.path.join('..', 'data', 'df_libor.pkl'))

             Swap1Y  Swap2Y  Swap3Y  Swap4Y  Swap5Y  Swap7Y  Swap10Y  Swap30Y  \
Time Period                                                                     
2000-07-03     7.10    7.16    7.17    7.17    7.17    7.20     7.24     7.24   
2000-07-05     7.03    7.06    7.07    7.07    7.08    7.11     7.14     7.16   
2000-07-06     7.07    7.13    7.14    7.15    7.16    7.19     7.21     7.21   
2000-07-07     7.01    7.04    7.06    7.06    7.07    7.10     7.14     7.14   
2000-07-10     7.04    7.09    7.11    7.13    7.14    7.17     7.20     7.19   
...             ...     ...     ...     ...     ...     ...      ...      ...   
2011-12-14     0.70    0.72    0.82    1.02    1.26    1.68     2.08     2.66   
2011-12-15     0.68    0.72    0.82    1.03    1.25    1.67     2.06     2.62   
2011-12-16     0.68    0.70    0.79    1.00    1.23    1.64     2.04     2.61   
2011-12-19     0.70    0.74    0.83    1.02    1.25    1.64     2.02     2.58   
2011-12-20     0.69    0.74 